In [ ]:
import pandas as pd, re, nltk, numpy as np, string
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB

In [2]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_features.csv', encoding='utf-8')
df.head()

,Unnamed: 0,source,no_words_inSent_SS,no_words_inSent_SK,sim_sent
0,0,Minghella Sohn italienisch-schottischer Eltern...,"[[1], [0], [0], [0], [0], [0], [0], [0], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[1], [0], [0], [0], [0], [0], [1], [0], [0], ..."
1,1,Ende 1940er Jahre wurde erste Auteur-Theorie f...,"[[2], [1], [0], [1], [0], [1], [0], [2], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0], [0], [0], [0], [0], [1], [1], [1], [1], ..."
2,2,"Al Pacino , geboren Manhattan , Sohn Salvatore...","[[4], [0], [2], [1], [2], [1], [0], [2], [4], ...","[[0], [0], [0], [1], [1], [0], [0], [0], [0], ...","[[1], [0], [1], [1], [1], [1], [0], [0], [0], ..."
3,3,Der Name Alkalimetalle leitet arabischen Wort ...,"[[1], [1], [0], [0], [1], [2], [1], [1], [1], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [1], ..."
4,4,Die Arbeit bereits seit Altertum Gegenstand re...,"[[0], [0], [0], [0], [0], [0], [0], [2], [0], ...","[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[1], [1], [0], [1], [0], [0], [0], [0], [1], ..."


In [5]:
stopword_list = nltk.corpus.stopwords.words('german')

def remove_stopwords(text):
    filtered_words = [word for word in nltk.word_tokenize(text) if word not in stopword_list]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

df["source"] = df["source"].apply(remove_stopwords)

In [6]:
from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')

cv = CountVectorizer(max_df=0.85,stop_words=german_stop_words, max_features=10000)
word_count_vector = cv.fit_transform(df['source'])

In [7]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [8]:
tfidf_transformer.idf_
feature_names=cv.get_feature_names()

In [9]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [10]:
doc = df['source'][2568]
def get_kew(df):
    kw=[]
    tf_idf_vector=tfidf_transformer.transform(cv.transform([df]))
    sorted_items=sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_from_vector(feature_names,sorted_items,10)
    for k in keywords:
        kw.append(k)
    return kw

df['keywords'] = df['source'].apply(get_kew)

In [11]:
#@title Default title text
def capital_lWords(df):
    tokens = sent_tokenize(df, language="german")
    return list([list(sum([c.isupper() for c in a])) for a in tokens])

def W_sourceSummary(df, df2):
    f = []
    text2 = df2
    text2 = text2.translate(str.maketrans('', '', string.punctuation))
    tokens = sent_tokenize(df, language = "german")
    tok2 = word_tokenize(text2, language = "german")
    for s in tokens:
        f.append(len(list((set(s.split()).intersection(set(tok2))))))
    return [[el] for el in f]

def W_sourceKeywords(df, df2):
    f = []
    df.lower()
    text2 = df2
    text2 = text2.translate(str.maketrans('', '', string.punctuation))
    tokens = sent_tokenize(df, language = "german")
    tok2 = word_tokenize(text2, language = "german")
    for s in tokens:
        f.append(len(list((set(s.split()).intersection(set(tok2))))))
    return [[el] for el in f]

def listToString(df):
    string = ' '.join([str(e) for e in df])
    return string

In [12]:
df['no_words_inSent_SS'] = df.apply(lambda x: W_sourceSummary(x['source'], x['summary']), axis=1)
df['keywords'] = df['keywords'].apply(listToString)
df['no_words_inSent_SK'] = df.apply(lambda x: W_sourceKeywords(x['source'], x['keywords']), axis=1)

In [ ]:
df.no_words_inSent_SK[0]

In [13]:
nltk.download('wordnet') # first-time use only
german_stop_words = stopwords.words('german')
lemmer = nltk.stem.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=german_stop_words)
def cos_similarity(textlist):
    tfidf = TfidfVec.fit_transform(textlist)
    return (tfidf * tfidf.T).toarray()

def get_docs(df):
    threshold = 0.15
    doc = sent_tokenize(df, language="german")
    lst, tags, output_tuples, similar_sent = [], [], [], [] 
    arr = cos_similarity(doc)
    doc_names = ["s"+str(s) for s in range(0, len(doc), 1)]
    for row in range(len(arr)):
        lst = [row+1+idx for idx, num in \
                  enumerate(arr[row, row+1:]) if num >= threshold]
        [(output_tuples.append((doc[row], doc[item]))) for item in lst]
    for tup in output_tuples:
        for sent in tup:
            similar_sent.append(sent)
    for sent in doc:
        if sent in set(doc).intersection(set(similar_sent)):
            tags.append(1)
        else:
            tags.append(0)
    return [[el] for el in tags]
#get_docs(df['source'][0])
df['sim_sent'] = df['source'].apply(get_docs)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dorian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
H:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['al', 'au', 'bi', 'de', 'diesis', 'dy', 'e', 'mus', 'un', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [ ]:
dat = df.drop(['summary', 'keywords'], axis=1)

In [ ]:
dat.to_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_features.csv')

In [ ]:
y_train = np.array(["new york is a hell of a town",
                    "new york was originally dutch",
                    "new york is also called the big apple",
                    "nyc is nice",
                    "the capital of great britain is london. london is a huge metropolis which has a great many number of people living in it. london is also a very old town with a rich and vibrant cultural history.",
                    "london is in the uk. they speak english there. london is a sprawling big city where it's super easy to get lost and i've got lost many times.",
                    "london is in england, which is a part of great britain. some cool things to check out in london are the museum and buckingham palace.",
                    "london is in great britain. it rains a lot in britain and london's fogs are a constant theme in books based in london, such as sherlock holmes. the weather is really bad there.",])
type(y_train)
y_train.shape()

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import FunctionTransformer

X_train = np.array(list(df.source[0]))
y_train = np.array(df.sim_sent[0])

X_test = np.array([sent_tokenize(df.source[5], language='german')])   


def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=1,max_df=2,lowercase=False)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ]))
    ])),
    ('clf', OneVsRestClassifier(LinearSVC()))])

classifier.fit(X_train, y_train)
predicted = classifier.predict(X_test)
predicted

In [5]:
data = df.head(10000)

In [15]:
data = data.drop(['summary', 'keywords'], axis=1)

In [58]:
X,y = data.source, data.sim_sent

In [50]:
def identify_tokens(row):
    tokens = sent_tokenize(row, language='german')
    return tokens
X = X.apply(identify_tokens)
y = y.apply(identify_tokens)

In [51]:
X = np.array(X)
y= np.array(y)

In [8]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('german'))
X = tfidfconverter.fit_transform(documents).toarray()

In [ ]:
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

In [ ]:
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

cls1 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(min_df=5, max_df=0.7, stop_words=stopwords.words('german'),lowercase=False)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
    ])),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=0))])
cls3.fit(X_train, y_train)

In [60]:
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

cls1 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(max_features=500, min_df=5, max_df=0.7, stop_words=stopwords.words('german'),lowercase=False)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
    ])),
    ('clf', svm.SVC())])
cls1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text',
                                                 Pipeline(steps=[('vectorizer',
                                                                  CountVectorizer(lowercase=False,
                                                                                  max_df=0.7,
                                                                                  max_features=500,
                                                                                  min_df=5,
                                                                                  stop_words=['aber',
                                                                                              'alle',
                                                                                              'allem',
                                                                                              'allen',
                                       

In [62]:
def get_text_length(x):
    return np.array([len(t) for t in x]).reshape(-1, 1)

cls1 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('vectorizer', CountVectorizer(max_features=500, min_df=5, max_df=0.7, stop_words=stopwords.words('german'),lowercase=False)),
            ('tfidf', TfidfTransformer()),
        ])),
        ('length', Pipeline([
            ('count', FunctionTransformer(get_text_length, validate=False)),
        ])),
    ])),
    ('clf', MultinomialNB())])
cls2.fit(X_train, y_train)

NameError: name 'MultinomialNB' is not defined

In [ ]:
y_pred = cls1.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

In [ ]:
y_pred2 = cls2.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))
print(accuracy_score(y_test, y_pred2))

In [ ]:
y_pred3 = cls3.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,y_pred3))
print(classification_report(y_test,y_pred3))
print(accuracy_score(y_test, y_pred3))